In [12]:
import os
import glob
import plotly
import osmnx as ox
import plotly.express as px
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ipypb import track
from shapely.geometry import Polygon, MultiPolygon

In [11]:
Data_Path = "C:\\Users\\sahit\\ADES\\ADES\\DataSource"

In [14]:
bridges = pd.read_excel(Data_Path + '\\BMMS_overview_SortedRoad.xlsx')

Possible approaches - 
1. Check the difference in latitude and longitude and it must be meaningful 
2. Check the latitude difference with the chainage of the bridge part that is calculated. 
3. 

In [20]:
bridges.head(15)

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.50,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.40,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise
3,N1,8.976,PC Girder Bridge,LRP008b,KANCHPUR PC GIRDER BRIDGE,397.00,C,101102,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,14.65,1986.0,8.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702083,90.515917,bcs1
4,N1,10.543,Box Culvert,LRP010a,KATCHPUR BOX CULVERT,8.00,B,101106,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.543,21.70,1987.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.702056,90.528194,bcs1
5,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.88,12.20,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699833,90.530722,bcs1
6,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.20,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699667,90.530722,bcs1
7,N1,11.296,Box Culvert,LRP011a,NAYABARI KASPUR BOX CULVERT,8.30,A,101110,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.296,21.45,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.697361,90.533833,bcs1
8,N1,11.808,Box Culvert,LRP011c,NAYABARI BOX CULVERT,10.60,B,101114,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.808,22.40,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694306,90.537861,bcs1
9,N1,12.239,Box Culvert,LRP012a,KHAS PARA BOX CULVERT,9.30,A,101117,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,12.239,21.00,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.692333,90.541000,bcs1


In [19]:
bridges["type"].unique()

array(['Box Culvert', 'PC Girder Bridge', 'RCC Girder Bridge', 'PC Box',
       'Slab Culvert', 'RCC Bridge', 'Arch Masonry',
       'Truss With Timber Deck', 'Steel Beam & RCC Slab',
       'Baily with Steel Deck', 'Truss with RCC Slab',
       'Truss with Steel Deck', 'Baily with Timber Deck', 'Pipe Culvert'],
      dtype=object)